<a href="https://colab.research.google.com/github/zahidmb/DL/blob/main/Simple_Classification_wit_FL_on_Cifar10_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Classification wit FL on Cifar10 dataset


In [ ]:
import numpy as np
import keras
from keras import layers


## Dataset

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# # Make sure images have shape (28, 28, 1)
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

### Splitting training data for multiple clients

In [ ]:
x_train_splits = np.array_split(x_train,4)
y_train_splits = np.array_split(y_train,4)

In [ ]:
x_train_splits[1].shape
# len(x_train_splits)

(12500, 32, 32, 3)

## Model Designing

In [ ]:
def create_model():
  model = keras.Sequential(
      [
          keras.Input(shape=input_shape),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(num_classes, activation="softmax"),
      ]
  )

  # model.summary()
  return model


## Creating multiple clients (models)

In [ ]:
# model_1 = create_model()
# model_1.summary()
model = []
for i in range(4):
  model.append(create_model())
  # model_i.summary()
  # print(i)

In [ ]:
model[3].summary()

## Compiling models

In [ ]:
for i in range(4):
  model[i].compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

## Training models

In [ ]:
batch_size = 128
epochs = 5
for i in range(4):
  print("Training %s model :" % (i))
  model[i].fit(x_train_splits[i], y_train_splits[i], validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)

# model[0].fit(x_train_splits[0], y_train_splits[0], validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)
# model[1].fit(x_train_splits[1], y_train_splits[1], validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)

Training 0 model :
Epoch 1/5
98/98 [==============================] - 2s 14ms/step - loss: 2.0024 - accuracy: 0.2730 - val_loss: 1.7265 - val_accuracy: 0.3985
Epoch 2/5
98/98 [==============================] - 1s 7ms/step - loss: 1.6680 - accuracy: 0.3997 - val_loss: 1.5712 - val_accuracy: 0.4458
Epoch 3/5
98/98 [==============================] - 1s 8ms/step - loss: 1.5389 - accuracy: 0.4486 - val_loss: 1.4590 - val_accuracy: 0.4907
Epoch 4/5
98/98 [==============================] - 1s 8ms/step - loss: 1.4435 - accuracy: 0.4905 - val_loss: 1.3883 - val_accuracy: 0.5157
Epoch 5/5
98/98 [==============================] - 1s 8ms/step - loss: 1.3771 - accuracy: 0.5161 - val_loss: 1.3451 - val_accuracy: 0.5228
Training 1 model :
Epoch 1/5
98/98 [==============================] - 3s 12ms/step - loss: 2.0101 - accuracy: 0.2678 - val_loss: 1.7295 - val_accuracy: 0.3914
Epoch 2/5
98/98 [==============================] - 1s 8ms/step - loss: 1.6822 - accuracy: 0.3966 - val_loss: 1.5603 - val_accu

In [ ]:

score = model[0].evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Test loss: 1.3531177043914795
Test accuracy: 0.5239999890327454


## Taking aggregation

In [ ]:
predictions = []
for i in range(4):

  predictions.append(model[i].predict(x_test))


# predictions[0]

313/313 [==============================] - 1s 3ms/step


In [ ]:
# w = model[0].get_layer("dense").get_weights()[0] # weights
w0 = model[0].get_weights()
w1 = model[1].get_weights()
w2 = model[2].get_weights()
w3 = model[3].get_weights()

In [ ]:
pairwise_dist = np.vstack((w0,w1,w2,w3))

/usr/local/lib/python3.10/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [ ]:
avg_w = np.average(pairwise_dist,axis=0)
avg_w[0].shape

(3, 3, 3, 32)

## Complete Code

In [1]:
import numpy as np
import keras
from keras import layers

# Model / data parameters
num_classes = 10
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Splitting training data for multiple clients
x_train_splits = np.array_split(x_train,4)
y_train_splits = np.array_split(y_train,4)


#Model Designing
def create_model():
  model = keras.Sequential(
      [
          keras.Input(shape=input_shape),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(num_classes, activation="softmax"),
      ]
  )

  # model.summary()
  return model

# Creating multiple clients (models)
model = []
for i in range(4):
  model.append(create_model())

# Compiling models
for i in range(4):
  model[i].compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training models
batch_size = 128
epochs = 5
for i in range(4):
  print("Training %s model :" % (i))
  model[i].fit(x_train_splits[i], y_train_splits[i], validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)

# Taking aggregation
w0 = model[0].get_weights()
w1 = model[1].get_weights()
w2 = model[2].get_weights()
w3 = model[3].get_weights()

pairwise_dist = np.vstack((w0,w1,w2,w3))

avg_w = np.average(pairwise_dist,axis=0)

print("Average weights:")
avg_w.shape

170498071/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Training 0 model :
Epoch 1/5
98/98 [==============================] - 6s 16ms/step - loss: 2.0033 - accuracy: 0.2722 - val_loss: 1.7271 - val_accuracy: 0.3846
Epoch 2/5
98/98 [==============================] - 1s 8ms/step - loss: 1.6513 - accuracy: 0.4144 - val_loss: 1.5955 - val_accuracy: 0.4355
Epoch 3/5
98/98 [==============================] - 1s 8ms/step - loss: 1.5201 - accuracy: 0.4538 - val_loss: 1.4746 - val_accuracy: 0.4798
Epoch 4/5
98/98 [==============================] - 1s 8ms/step - loss: 1.4402 - accuracy: 0.4822 - val_loss: 1.4012 - val_accuracy: 0.5068
Epoch 5/5
98/98 [==============================] - 1s 9ms/step - loss: 1.3875 - accuracy: 0.5091 - val_loss: 1.3792 - val_accuracy: 0.5172
Training 1 model :
Epoch 1/5
98/98 [==============================] - 2s 14ms/step - loss: 2.0045 - accuracy: 0.2698 - val_loss: 1.7378 - val_acc

/usr/local/lib/python3.10/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


(6,)